In [ ]:
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

%load_ext autoreload
%autoreload 2

from fdshapley import FederatedShapley


import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [ ]:
# Load data from https://www.openml.org/d/554
X, y = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False)

In [ ]:
def split(X, y, n_participants, frac_test=0.2):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2) 
    xs_train, ys_train = np.split(X_train, n_participants), np.split(y_train, n_participants)
    return  xs_train, ys_train, X_test, y_test

In [ ]:
xs_train, ys_train, X_test, y_test = split(X, y, 20)

In [ ]:
params = {
    'C': 50/len(ys_train[0]), 
    'max_iter': 1, 
    'penalty':"l1",
    'solver':"saga",
    'warm_start': True,
    'fit_intercept':False
}

In [ ]:
data_train = [[x,y] for x,y in zip(xs_train, ys_train)]
data_test = [X_test, y_test]

for i in [0,1,2,3]:
    data_train[i][1] = np.random.randint(0,10, len(data_train[i][1]))

In [ ]:
fed = FederatedShapley(data_train, data_test, params)
res = fed.federatedSVEstimation(0.1, 20)
fed.clf.coef_ = fed.w
fed.clf.score(X_test, y_test)

In [ ]:
plt.bar(range(20), res)